In [1]:
import pandas as pd

# Load CSV without header
df = pd.read_csv("PS_2026.01.16_22.02.52.csv", header=None)

# Set row 295 as column names
df.columns = df.iloc[295]

# Drop metadata + header rows
df = df.iloc[296:].reset_index(drop=True)

# Verify
df.head()
df.shape


ParserError: Error tokenizing data. C error: Expected 1 fields in line 296, saw 289


In [2]:
import pandas as pd

df = pd.read_csv(
    "PS_2026.01.16_22.02.52.csv",
    comment="#"
)

df.head()
df.shape
#metadata rows removed , correct headers , and stable dataframe

C:\Users\aryak\AppData\Local\Temp\ipykernel_2532\48136616.py:3: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


(39251, 289)

In [3]:
df.columns[:10]


Index(['rowid', 'pl_name', 'hostname', 'pl_letter', 'hd_name', 'hip_name',
       'tic_id', 'gaia_dr2_id', 'gaia_dr3_id', 'default_flag'],
      dtype='object')

In [4]:
required_cols = [
    # Planetary
    "pl_rade",      # Planet radius (Earth radii)
    "pl_masse",     # Planet mass (Earth masses)
    "pl_orbper",    # Orbital period (days)
    "pl_orbsmax",   # Semi-major axis (AU)
    "pl_eqt",       # Equilibrium temperature (K)
    "pl_dens",      # Planet density (g/cm^3)

    # Stellar
    "st_teff",      # Star effective temperature (K)
    "st_lum",       # Star luminosity (log solar)
    "st_met",       # Star metallicity
    "st_spectype"   # Star spectral type
]


In [5]:
df_hab = df[required_cols]


In [6]:
df_hab.head()
df_hab.shape
df_hab.isnull().sum()


pl_rade        12200
pl_masse       34480
pl_orbper       3344
pl_orbsmax     17292
pl_eqt         22032
pl_dens        36523
st_teff         3529
st_lum         29593
st_met         14469
st_spectype    36352
dtype: int64

In [7]:
# Separate numerical and categorical columns
num_cols = df_hab.select_dtypes(include="number").columns
cat_cols = ["st_spectype"]

# Median imputation for numerical features
df_hab[num_cols] = df_hab[num_cols].fillna(df_hab[num_cols].median())

# Mode imputation for star spectral type
df_hab["st_spectype"] = df_hab["st_spectype"].fillna(
    df_hab["st_spectype"].mode()[0]
)


C:\Users\aryak\AppData\Local\Temp\ipykernel_2532\2740051212.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hab[num_cols] = df_hab[num_cols].fillna(df_hab[num_cols].median())
C:\Users\aryak\AppData\Local\Temp\ipykernel_2532\2740051212.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hab["st_spectype"] = df_hab["st_spectype"].fillna(


In [8]:
df_hab = df_hab.copy()


In [9]:
num_cols = df_hab.select_dtypes(include="number").columns

df_hab.loc[:, num_cols] = df_hab.loc[:, num_cols].fillna(
    df_hab[num_cols].median()
)


In [10]:
df_hab.loc[:, "st_spectype"] = df_hab["st_spectype"].fillna(
    df_hab["st_spectype"].mode()[0]
)



In [11]:
df_hab.isnull().sum()



pl_rade        0
pl_masse       0
pl_orbper      0
pl_orbsmax     0
pl_eqt         0
pl_dens        0
st_teff        0
st_lum         0
st_met         0
st_spectype    0
dtype: int64

In [12]:
df_hab = df_hab[
    (df_hab["pl_rade"] > 0) &          # radius must be positive
    (df_hab["pl_masse"] > 0) &         # mass must be positive
    (df_hab["pl_eqt"] > 0) &           # temperature in Kelvin
    (df_hab["pl_orbper"] > 0) &        # orbital period > 0
    (df_hab["pl_orbsmax"] > 0)         # distance > 0
].reset_index(drop=True)


In [13]:
df_hab.shape


(39251, 10)

In [14]:
import numpy as np

num_cols = df_hab.select_dtypes(include="number").columns

Q1 = df_hab[num_cols].quantile(0.25)
Q3 = df_hab[num_cols].quantile(0.75)
IQR = Q3 - Q1

df_hab = df_hab[~(
    (df_hab[num_cols] < (Q1 - 1.5 * IQR)) |
    (df_hab[num_cols] > (Q3 + 1.5 * IQR))
).any(axis=1)].reset_index(drop=True)


In [15]:
df_hab.shape


(6637, 10)

In [16]:
df_hab.to_csv("step4_clean.csv", index=False)


In [17]:
import pandas as pd

df_hab = pd.read_csv("step4_clean.csv")
df_hab.shape


(6637, 10)

In [18]:
# Rebuild Step-4 clean dataset from df
df_hab = df[[
    "pl_rade", "pl_masse", "pl_orbper", "pl_orbsmax",
    "pl_eqt", "pl_dens",
    "st_teff", "st_lum", "st_met", "st_spectype"
]].copy()


In [19]:
num_cols = df_hab.select_dtypes(include="number").columns

df_hab.loc[:, num_cols] = df_hab[num_cols].fillna(
    df_hab[num_cols].median()
)

df_hab.loc[:, "st_spectype"] = df_hab["st_spectype"].fillna(
    df_hab["st_spectype"].mode()[0]
)


In [20]:
df_hab = df_hab[
    (df_hab["pl_rade"] > 0) &
    (df_hab["pl_masse"] > 0) &
    (df_hab["pl_eqt"] > 0) &
    (df_hab["pl_orbper"] > 0) &
    (df_hab["pl_orbsmax"] > 0)
].reset_index(drop=True)

df_hab.shape


(39251, 10)

In [21]:
df_hab.to_csv("step4_clean.csv", index=False)


In [22]:
iqr_cols = ["pl_rade", "pl_eqt", "pl_orbsmax"]

Q1 = df_hab[iqr_cols].quantile(0.25)
Q3 = df_hab[iqr_cols].quantile(0.75)
IQR = Q3 - Q1

df_hab = df_hab[~(
    (df_hab[iqr_cols] < (Q1 - 1.5 * IQR)) |
    (df_hab[iqr_cols] > (Q3 + 1.5 * IQR))
).any(axis=1)].reset_index(drop=True)

df_hab.shape


(16342, 10)

In [23]:
df_hab.to_csv("step5_clean.csv", index=False)


In [24]:
# Extract main spectral class (G, K, M, F, etc.)
df_hab["st_type_main"] = df_hab["st_spectype"].str[0]

# One-hot encode
df_hab = pd.get_dummies(
    df_hab,
    columns=["st_type_main"],
    drop_first=True
)

# Drop original spectral type column
df_hab.drop(columns=["st_spectype"], inplace=True)

df_hab.shape


(16342, 14)

In [25]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df_scaled = df_hab.copy()

df_scaled[:] = scaler.fit_transform(df_scaled)

df_scaled.shape


ModuleNotFoundError: No module named 'sklearn'

In [26]:
!pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.0 MB 2.8 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/8.0 MB 1.5 MB/s eta 0:00:05
   ----- ---------------------------------- 1.0/8.0 MB 1.3 MB/s eta 0:00:06
   ----- ---------------------------------- 1.0/8.0 MB 1.3 MB/s eta 0:00:06
   ------- -------------------------------- 1.6/8.0 MB 1.4 MB/s eta 0:00:05
   ---------- ----------------------------- 2.1/8.0 MB 1.5 MB/s eta 0:00:04
   ------------- -------------------------- 2.6/8.0 MB 1.7 MB/s eta 0:00:04
   --------------- ------------------------ 3.1/8.0 MB 1.8 MB/s eta 0:00:03
   ------------------ --------------------- 3.7/8.0 MB 1.9 MB/s eta 0:00:03
   -------------------- ------------------- 4.2/8.0 MB 2.0 MB/s eta 0:00:02
   ----------------------- ---------------- 4.7/8.0 MB 2.0 MB/s eta 0:00:02
   --------------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
from sklearn.preprocessing import StandardScaler


In [28]:
scaler = StandardScaler()

df_scaled = df_hab.copy()

df_scaled[:] = scaler.fit_transform(df_scaled)

df_scaled.shape



C:\Users\aryak\AppData\Local\Temp\ipykernel_2532\3369130025.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.01106341 -0.01106341 -0.01106341 ... -0.01106341 -0.01106341
 -0.01106341]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_scaled[:] = scaler.fit_transform(df_scaled)
C:\Users\aryak\AppData\Local\Temp\ipykernel_2532\3369130025.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.09721554 -0.09721554 -0.09721554 ... -0.09721554 10.28642103
 -0.09721554]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df_scaled[:] = scaler.fit_transform(df_scaled)
C:\Users\aryak\AppData\Local\Temp\ipykernel_2532\3369130025.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.17151479  0.17

(16342, 14)

In [29]:
from sklearn.preprocessing import StandardScaler

# Identify continuous (non one-hot) columns
continuous_cols = [
    "pl_rade", "pl_masse", "pl_orbper", "pl_orbsmax",
    "pl_eqt", "pl_dens",
    "st_teff", "st_lum", "st_met"
]

# Create a copy
df_scaled = df_hab.copy()

# Scale only continuous columns
scaler = StandardScaler()
df_scaled[continuous_cols] = scaler.fit_transform(df_scaled[continuous_cols])

df_scaled.shape


(16342, 14)

In [31]:
df_scaled.to_csv("preprocessed.csv", index=False)
